In [5]:
import pandas as pd
import numpy as np


from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs


from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()
# load in params and websites
webs_paras = pd.read_csv('../data/normalized.csv')
webs = webs_paras.drop(['(Doc complete) Byets in','(Fully loaded) Bytes in','(Fully loaded) Requests','(Fully loaded) Requests','(Doc complete) Requests'], axis = 1)
webs_requests = webs_paras[['Site name','(Fully loaded) Requests','(Fully loaded) Requests','(Doc complete) Requests']]



 #webs['(Doc complete) Byets in'].describe()

list_of_params = list(webs.columns[1:].unique())

list_of_params.sort()

Loading BokehJS ...

In [142]:
def make_dataset(param_list, range_start, range_end, bandwidth):

    xs = []
    ys = []
    colors = []
    labels = []

    for i, param in enumerate(param_list):
        subset = webs[param]
        print i, param
        if param in webs.columns:
            print "works"
        #subset = subset[webs[param]]
        
        print subset

        kde = gaussian_kde(webs[param], bw_method=bandwidth)

        # Evenly space x values
        x = np.linspace(range_start, range_end, 100)

        y = kde.pdf(x)

        # Append the values to plot
        xs.append(list(x))
        ys.append(list(y))

        # Append the colors and label
        colors.append(para_colors[i])
        labels.append(param)

    new_src = ColumnDataSource(data={'x':xs, 'y':ys,
                                    'color':colors, 'label':labels})
    return new_src


In [143]:
make_dataset(webs.columns[1:], 0,1,0.005)

0 (Fully loaded) Time
works
0     0.176256
1     0.285687
2     0.295591
3     0.050200
4     0.093563
5     0.561896
6     0.048314
7     0.064136
8     0.065763
9     0.156520
10    0.341712
11    0.052700
12    0.086088
13    0.523627
14    0.132304
15    0.168121
16    0.022872
17    0.063523
18    0.062226
19    0.063971
20    0.073237
21    0.076822
22    0.037373
23    0.202853
24    0.045296
25    0.503726
26    0.044188
27    0.362202
28    0.210398
29    1.000000
30    0.977930
31    0.129238
32    0.027611
33    0.000000
34    0.227776
35    0.351945
36    0.047819
37    0.735133
38    0.113629
39    0.017944
40    0.176279
41    0.020207
42    0.085239
43    0.058170
44    0.089531
45    0.079698
46    0.306862
47    0.246121
48    0.235534
Name: (Fully loaded) Time, dtype: float64


NameError: global name 'para_colors' is not defined

In [151]:
# %load ../scripts/density.py
import pandas as pd
import numpy as np

from scipy.stats import gaussian_kde

from bokeh.plotting import figure
from bokeh.models import (CategoricalColorMapper, HoverTool,
                          ColumnDataSource, Panel,
                          FuncTickFormatter, SingleIntervalTicker,LinearAxis)
from bokeh.models.widgets import (CheckboxGroup, Slider, RangeSlider,
                                  Tabs, CheckboxButtonGroup,
                                  TableColumn, DataTable, Select)
from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

def density_tab(webs):

    def make_dataset(param_list, range_start, range_end, bandwidth):

        xs = []
        ys = []
        colors = []
        labels = []

        for i, param in enumerate(param_list):
            subset = webs[param]
            
            subset = subset[webs[param].between(range_start,range_end)]
           

            kde = gaussian_kde(webs[param], bw_method=bandwidth)

            # Evenly space x values
            x = np.linspace(range_start, range_end, 100)

            y = kde.pdf(x)

            # Append the values to plot
            xs.append(list(x))
            ys.append(list(y))

            # Append the colors and label
            colors.append(para_colors[i])
            labels.append(param)

        new_src = ColumnDataSource(data={'x':xs, 'y':ys,
                                        'color':colors, 'label':labels})
        return new_src

    def make_plot(src):
        p = figure(plot_width = 700, plot_height = 700, 
                    title = 'Density Plot of Arrival Delays by Airline',
                    x_axis_label = 'Delay (min)', y_axis_label = 'Density')
        p.multi_line('x', 'y', color = 'color', legend = 'label',
                    line_width = 3, source = src)
        hover = HoverTool(tooltips=[('parameter','@label'),],
                          line_policy = 'next')

        p.add_tools(hover)

        p = style(p)
        return p

    def update(attr, old, new):

        parameters_to_plot = [param_selection.labels[i] for i in
                              param_selection.active]
        if bandwidth_choose.active == []:
            bandwidth = None
        else:
            bandwidth = bandwidth_select.value

        new_src = make_dataset(parameters_to_plot,
                                    range_start = range_select.value[0],
                                    range_end = range_select.value[1],
                                    bandwidth = bandwidth)
        src.data.update(new_src.data)

    def style(p):

        # Title
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'
        return p

    # Parameter and colors
    available_parameters = list(set(webs.columns[1:]))
    #print available_parameters
    available_parameters.sort()

    para_colors = Category20_16
    para_colors.sort()

    param_selection = CheckboxGroup(labels = available_parameters,
                                    active = [0,1])
    param_selection.on_change('active', update)

    range_select = RangeSlider(start = 0, end = 1, value = (0,1),
                               step = 0.0025, title = 'Range of Parameter')
    range_select.on_change('value', update)

    initial_parameter = [param_selection.labels[i] for 
                        i in param_selection.active]
    # Bandwidth of kernel
    bandwidth_select = Slider(start = 0.1, end = 5,
                              step = 0.1, value = 0.5,
                              title = 'Bandwidth for Density Plot')
    bandwidth_select.on_change('value', update)

    bandwidth_choose = CheckboxButtonGroup(
                        labels = ['Choose Bandwidth (Else Auto)'], active = [])
    bandwidth_choose.on_change('active', update)

    # Make the density data source
    src = make_dataset(initial_parameter,
                        range_start = range_select.value[0],
                        range_end = range_select.value[1],
                        bandwidth = bandwidth_select.value)

    p = make_plot(src)

    p = style(p)

    controls = WidgetBox(param_selection, range_select,
                        bandwidth_select, bandwidth_choose)

    layout = row(controls, p)

    tab = Panel(child=layout, title = 'Density Plot')
    tabs = Tabs(tabs=[tab])
    
 


In [154]:
webs.columns[1:]

Index([u'(Fully loaded) Time', u'DOM elements', u'First byte', u'Load time',
       u'Speed Index', u'Start render'],
      dtype='object')

In [152]:
handler = FunctionHandler(density_tab)
app = Application(handler)


In [155]:
show(app, 'localhost:800')

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=6c38ae2f-f5c2-4302-b992-0308efad558b&bokeh-absolute-url=http://localhost:63199&resources=none (::1)
HTTPServerRequest(protocol='http', host='localhost:63199', method='GET', uri='/autoload.js?bokeh-autoload-element=6c38ae2f-f5c2-4302-b992-0308efad558b&bokeh-absolute-url=http://localhost:63199&resources=none', version='HTTP/1.1', remote_ip='::1', headers={'Accept-Language': 'en-US,en;q=0.9', 'Accept-Encoding': 'gzip, deflate, br', 'Host': 'localhost:63199', 'Accept': '*/*', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36', 'Connection': 'keep-alive', 'Referer': 'http://localhost:8888/notebooks/Networking/Webranking/bokeh_app/scripts/density.py.ipynb', 'Cookie': 'csrftoken=1lnVg33qIJ6oRSJGgGut0x5SpizJIWLAmAnAFVancy33zOMTr6aulabK1J6sREpA; _xsrf=2|d6c96b57|1874fd77593ecd886e2147e4d1fc715a|1538785729; username-localhost-8888="